##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Глубокая сверточная генеративная состязательная сеть(Deep Convolutional Generative Adversarial Network)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/dcgan">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/dcgan.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/dcgan.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/generative/dcgan.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве показано, как создавать изображения рукописных цифр с помощью [Deep Convolutional Generative Adversarial Network](https://arxiv.org/pdf/1511.06434.pdf)(DCGAN). Код написан с использованием [Keras Sequential API](https://www.tensorflow.org/guide/keras) с циклом обучения `tf.GradientTape`.

## Что такое GAN?

[Генеративные состязательные сети](https://arxiv.org/abs/1406.2661)(GAN) - одна из самых интересных идей в современной информатике. Две модели обучаются одновременно в состязательном процессе. *Генератор* («художник») учится создавать изображения, которые выглядят реальными, а *дискриминатор* («искусствовед») учится отличать реальные изображения от подделок.

![Схема генератора и дискриминатора](./images/gan1.png)

Во время обучения *генератор* постепенно становится лучше в создании изображений, которые выглядят реальными, в то время как *дискриминатор* учится лучше различать их. Процесс достигает равновесия, когда *дискриминатор* больше не может отличить реальные изображения от подделок.

![Вторая схема генератора и дискриминатора](./images/gan2.png)

Это руководство демонстрирует этот процесс, используя датасет MNIST. Следующая анимация показывает серию изображений, созданных *генератором*, когда он был обучен в течение 50 эпох. Изображения начинаются как случайный шум и со временем все больше напоминают рукописные цифры.

![пример вывода](https://tensorflow.org/images/gan/dcgan.gif)

Чтобы узнать больше о GAN, мы рекомендуем курс MIT [Intro to Deep Learning](http://introtodeeplearning.com/).

### Установка

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
# Для генерации GIF
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

### Загрузка и подготовка датасета

Для обучения генератора и дискриминатора вы будете использовать набор данных MNIST. Генератор будет генерировать рукописные цифры, похожие на данные MNIST.

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Нормализуем изображение до [-1, 1]

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [ ]:
# Разделяем на пакеты и перемешиваем
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Создание моделей

Обе модели, и генератор и дискриминатор, будут построены с помощью [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model).

### Генератор

Генератор использует слои `tf.keras.layers.Conv2DTranspose`(повышающая дискретизация) для создания изображения из начального числа(случайный шум). Начните со слоя `Dense`, который принимает это начальное значение в качестве входных данных, затем несколько раз увеличивайте разрешение, пока не достигнете желаемого размера изображения 28x28x1. Обратите внимание на активацию `tf.keras.layers.LeakyReLU` для каждого слоя, кроме выходного, который использует `tanh`.

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Примечание: None - это размер пакета

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

Используйте(пока еще не обученный) генератор для создания изображения.

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

### Дискриминатор

Дискриминатор - это классификатор изображений на основе CNN.

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

Используйте(пока еще не обученный) дискриминатор, чтобы классифицировать сгенерированные изображения как настоящие или поддельные. Модель будет обучена выводить положительные значения для реальных изображений и отрицательные значения для поддельных изображений.

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

## Определение функции потерь и оптимайзера

Определите функции потерь и оптимайзеры для обеих моделей.


In [ ]:
# метод возвращает вспомогательную функцию для вычисления потерь бинарной кросс-энтропии
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Потери дискриминатора

Этот метод определяет, насколько хорошо дискриминатор может отличать настоящие изображения от подделок. Он сравнивает предсказания дискриминатора на реальных изображениях с массивом единиц, а предсказания на поддельных(сгенерированных) изображениях - с массивом нулей.

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Потери генератора
Потери генератора показывают, насколько хорошо он смог обмануть дискриминатор. Интуитивно понятно, что если генератор работает хорошо, дискриминатор классифицирует поддельные изображения как реальные(или 1). Здесь мы сравним решения дискриминатора на сгенерированных изображениях с массивом единиц.

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

Оптимайзеры дискриминатора и генератора разные, поскольку мы будем обучать две сети отдельно.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### Сохранение контрольных точек
В этом руководстве также показано, как сохранять и восстанавливать модели, что может быть полезно в случае прерывания длительного обучающего процесса.

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Определение тренировочного цикла


In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# мы будем использовать один и тот же seed для обоих моделей
# поэтому создадит его отдельной переменной
seed = tf.random.normal([num_examples_to_generate, noise_dim])

Цикл обучения начинается с того, что генератор получает на вход случайное начальное число(seed). Это число используется для создания изображения. Затем дискриминатор используется для классификации реальных изображений(взятых из обучающего набора) и поддельных изображений(созданных генератором). Потери рассчитываются для каждой из этих моделей, а градиенты используются для обновления генератора и дискриминатора.

In [ ]:
# Обратите внимание на использование `tf.function`
# Эта аннотация заставляет функцию "компилироваться".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Создаем на лету изображения для генерации GIF
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Сохраняем модель каждые 15 эпох
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Генерируем после окончания последней эпохи
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

**Генерация и сохранение изображений**


In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Обратите внимание, что для параметра training установлено значение False.
  # Это значит, что все слои работают в режиме вывода(batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

## Обучение модели

Вызовите метод `train()`, определенный выше, для одновременного обучения генератора и дискриминатора. Учтите, что обучение GAN может оказаться непростым делом. Важно, чтобы генератор и дискриминатор не подавляли друг друга (например, чтобы они тренировались с одинаковой скоростью).

В начале тренировки сгенерированные изображения выглядят как случайный шум. По мере обучения сгенерированные цифры будут выглядеть все более реальными. Примерно через 50 эпох они напоминают цифры MNIST. Это может занять около одной минуты на эпоху с настройками по умолчанию в Colab.

In [ ]:
train(train_dataset, EPOCHS)

Восстановите последний чекпойнт

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Создание GIF


In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

Используйте `imageio` для создания анимированного gif из изображений, сохраненных во время обучения.

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

## Следующие шаги


В этом руководстве показан полный код, необходимый для написания и обучения GAN. В качестве следующего шага вы можете поэкспериментировать с другим набором данных, например, с набором данных Large-scale Celeb Faces Attributes (CelebA) [доступно на Kaggle](https://www.kaggle.com/jessicali9530/celeba-dataset) . Чтобы узнать больше о GAN, мы рекомендуем [Учебное пособие NIPS 2016: Генеративные состязательные сети](https://arxiv.org/abs/1701.00160).